This notebook will specifically use data from the Allen Insitute for Neural Dynamics and show how we can leverage CEBRA to analyse our in-house data. Here we will check if we can distinguish between trials where the mouse licked right vs trials where it licked left in the dynamic foraging task. We also tried to check if we can s 

In [38]:
import sys
import os # my addtion

import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
import cebra.data
import torch
import cebra.integrations
import cebra.datasets
from cebra import CEBRA
import torch
import pickle
import cebra_pack.cebra_utils as cp


from matplotlib.collections import LineCollection
import pandas as pd

## A. Load the Data

Here we load data from the Fibre Photometry pipeline of 4 Neuromodulators (DA, 5HT, ACh, NE) recorded in the Nucleus Acumbens region. The main neural data will be in the form of dF_F traces of these 4 Neuromodulators (NMs). These will be stored in a 2D array, 'all_nms'.

In [39]:
# load the dataframe that contains data from 1 session
df_trials_ses = pickle.load(open('../data/CO data/df.pkl', "rb"))

In [40]:
df_trials_ses.columns

Index(['bit_code', 'ses_idx', 'rpe', 'left_action_value', 'right_action_value',
       'licks L', 'licks R', 'Lick L (raw)', 'Lick R (raw)', 'trial', 'reward',
       'choice', 'go_cue_absolute_time', 'go_cue', 'choice_time',
       'reward_time', 'onset', 'NM', 'NM_name', 'region', 'last_value_NM',
       'overlap_index', 'NM_no_overlap', 'bins_mids', 'bins_mids_no_overlap'],
      dtype='object')

In [41]:
l1 = df_trials_ses['licks L'].iloc[:1765].values
l2 = df_trials_ses['licks L'].iloc[3530:5295].values
np.sum(l1-l2)

0.0

In [42]:
df_trials_ses[['licks L', 'licks R']]

,licks L,licks R
0,26.0,0.0
1,10.0,0.0
2,2.0,0.0
3,2.0,0.0
4,1.0,0.0
...,...,...
1760,1.0,0.0
1761,3.0,0.0
1762,2.0,0.0
1763,1.0,0.0


In [43]:
df_trials_ses[['licks L', 'licks R']].describe()

,licks L,licks R
count,7060.000000,7060.000000
mean,1.309915,2.229462
std,2.377764,3.430823
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,2.000000
max,26.000000,21.000000


In [44]:
np.unique(df_trials_ses['Lick L (raw)'].iloc[7], return_counts=True)

(array([0., 1.]), array([117,   1]))

NB: There's 1765 trials in this data frame (the indices are repeated over each neuromodulator)
all information is in the first 1765 rows so we'll consider those

In [45]:
n_trials = 1765

In [46]:
# download the dictionary containing the traces
traces = pickle.load(open('../data/CO data/traces.pkl', "rb"))

In [47]:
# load the trace times
trace_times = np.load('../data/CO data/Trace times.npy', allow_pickle=True)

In [48]:
# get the choice time 
choice_times = df_trials_ses['choice_time'][0:n_trials].to_numpy()

In [49]:
# Combine the traces into one 2D array
all_nms = np.array([traces[trace] for trace in traces.keys()])
all_nms = np.transpose(all_nms)

# change it to an array of floats (previously it was an array of object datatype)
all_nms_new = all_nms.astype(np.float64)
all_nms_new.shape


(218572, 4)

In [50]:
all_nms.shape

(218572, 4)

In [51]:
# change it to an array of floats (previously it was an array of object datatype)
all_nms_new = all_nms.astype(np.float64)
all_nms_new.shape

(218572, 4)

In [52]:
# convert it to a tensor (this is probably not necessary but we want it to be as close to the inputs in the previous notebook)
all_nms_tensor = torch.from_numpy(all_nms_new)
all_nms_tensor.shape

torch.Size([218572, 4])

## B. Format data and create the behavioural/auxiliary variables

Now let's format the data. We want to view the data in a 1 second window around the choice time at each trial in the session. The hope is that this will make it easy to identify the trials where it chose to lick left and those where it chose to lick right.

Each trial will be labelled as rewarded/unrewarded and this will be the behavioural variable we use for this analysis.

In [53]:
ins = np.argwhere(df_trials_ses['licks R'] > df_trials_ses['licks L'])

In [54]:
df_trials_ses['licks L'].to_numpy()

array([26., 10.,  2., ...,  2.,  1.,  1.])

In [55]:
444/(6616+444)

0.06288951841359773

In [56]:
6616+444

7060

6% of the trials had an equal number of left and right licks

In [57]:
indxs = np.argwhere(df_trials_ses['licks L'] == df_trials_ses['licks R'])

In [58]:
np.unique(df_trials_ses['licks L'].to_numpy()[indxs],return_counts=True)

(array([0., 1., 2.]), array([184, 240,  20]))

For the trials where the number of left and right trials are equal, the number of licks in either direction is either 0,1 or 2.

In [59]:
# Make a function to format the NM data into a 1s window around the choice

def format_data(neural_data, df, trace_times_, choice_times_ , window=None , window_size=10, n_trials=1765):

    # define the number of trials where the mouse made a choice
    n_choice_trials = np.unique(np.isnan(choice_times_),return_counts=True)[1][0]

    # list to hold all the 1s windows
    n_data_window = []

    # new trial label
    trial_labels = []

    # magnitude label (number of licks)
    n_licks = []

    # loop over all trials
    for i in range(0,n_trials):

        # skip trials where the animal didn't make a choice (null choice time)
        if np.isnan(choice_times_[i]):
            continue

        # find the index of the closest time to the choice time in the trace_times array 
        idx = np.abs(trace_times_ - choice_times_[i]).argmin()

        # take the previous 10 and/or the next 10 values of the NM data at these indices - 1s window
        if window =='before':
            n_data_window.append(neural_data[idx-10:idx])

        if window == 'after':
            n_data_window.append(neural_data[idx:idx+10])

        if window == None:
            n_data_window.append(neural_data[idx-10:idx+10])

        # label the timepoints as left or right choice
        if df['licks L'].iloc[i] >= df['licks R'].iloc[i]:
            # new trial label
            trial_labels.append(1)
            n_licks.append(df['licks L'].iloc[i])

        elif df['licks R'].iloc[i] > df['licks L'].iloc[i]:
            # new trial label
            trial_labels.append(0)
            n_licks.append(df['licks R'].iloc[i])


    # stack the nm data for each trial
    nms_HD = np.stack(n_data_window).reshape((n_choice_trials,-1))
    # format it into a tensor
    nms_HD = torch.from_numpy(nms_HD.astype(np.float64))
    print("neural tensor shape: ", nms_HD.shape)

    # convert trial labels into an array
    choice_labels = np.array(trial_labels)
    n_licks = np.array(n_licks)
    print("labels shape: ",choice_labels.shape)
    print("number of licks:", n_licks.shape)

    return nms_HD, choice_labels, n_licks

In [60]:
formatted_nms, trial_labels, n_licks = format_data(all_nms,df=df_trials_ses,trace_times_=trace_times, choice_times_=choice_times)

neural tensor shape:  torch.Size([1717, 80])
labels shape:  (1717,)
number of licks: (1717,)


In [61]:
plt.plot(n_licks)

In [62]:
# let's make a behaviour label that gives information on both the direction and how many licks in that direction it took
# -- use the 3D array like they did in the hippocampus demo

# but first let's start with just whether it licked left or right

Out of the 1765 trials in the session, this particular mouse made a choice in 1717 of them. So we will only use those trials. Therefore we have a binary array of shape (1717,) as our behavioural label. The NM data is organized such that we have 20 timesteps for each of the 4 neuromodulators in this 1 second window.

## C. Build and train the CEBRA models

In [63]:
# set the maximum number of iterations for training the model
max_iterations = 2000

In [64]:
# build a CEBRA-Time and CEBRA-Behaviour model
cebra_time_model = CEBRA(model_architecture='offset10-model-mse',
                        batch_size=512,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=3,
                        max_iterations=max_iterations,
                        distance='euclidean',
                        conditional='time',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

In [65]:
cebra_behaviour_model = CEBRA(model_architecture='offset10-model-mse',
                        batch_size=512,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=3,
                        max_iterations=max_iterations,
                        distance='euclidean',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

Train the two models

In [66]:
# train the time model (no labels here)
cebra_time_model.fit(formatted_nms)

pos:  0.8810 neg:  2.8742 total:  3.7552 temperature:  1.0000: 100%|██████████| 2000/2000 [00:12<00:00, 158.25it/s]


CEBRA(batch_size=512, conditional='time', distance='euclidean',
      max_iterations=2000, model_architecture='offset10-model-mse',
      output_dimension=3, temperature=1, time_offsets=10, verbose=True)

In [67]:
# train the behaviour model (use the labels here)
cebra_behaviour_model.fit(formatted_nms, trial_labels)

pos:  0.3213 neg:  5.7475 total:  6.0687 temperature:  1.0000: 100%|██████████| 2000/2000 [00:13<00:00, 148.18it/s]


CEBRA(batch_size=512, conditional='time_delta', distance='euclidean',
      max_iterations=2000, model_architecture='offset10-model-mse',
      output_dimension=3, temperature=1, time_offsets=10, verbose=True)

## D. Compute and view embeddings

Here, we compute the embeddings from the two trained models and then plot them.

In [68]:
time_embedding = cebra_time_model.transform(formatted_nms)

In [69]:
behaviour_embedding = cebra_behaviour_model.transform(formatted_nms)

In [70]:
# divide the labels into right and left
left = trial_labels==1
right = trial_labels==0

left = left.flatten()
right = right.flatten()

In [71]:
# sanity check - confirm that there's only two possibilities and that they add up to the total # of trials
print('Unique values (and their corresponding counts in the labels:',np.unique(right, return_counts=True))

print('Embedding (time) shape:',time_embedding.shape)
print('Embedding (behaviour) shape:',behaviour_embedding.shape)

print("Difference between the length of embedding array and the total number of trials:",np.sum(time_embedding.shape[0] - np.sum(np.unique(right, return_counts=True)[1])))

Unique values (and their corresponding counts in the labels: (array([False,  True]), array([ 713, 1004]))
Embedding (time) shape: (1717, 3)
Embedding (behaviour) shape: (1717, 3)
Difference between the length of embedding array and the total number of trials: 0


In [72]:
# get auc scores
mean_scores, errors = cp.get_auc([behaviour_embedding, time_embedding], trial_labels=trial_labels)
mean_scores

[0.8722473360415282, 0.5408450350072361]

In [73]:
# create a figure and make the plots
fig1 = plt.figure(figsize=(16,4))
gs = gridspec.GridSpec(1, 2, figure=fig1)

ax1 = fig1.add_subplot(gs[0,0], projection='3d')
ax2 = fig1.add_subplot(gs[0,1], projection='3d')
axes =[ax1,ax2]

for ax in axes:


    ax.set_xlabel("latent 1", labelpad=0.01)
    ax.set_ylabel("latent 2", labelpad=0.01)
    ax.set_zlabel("latent 3", labelpad=0.01)

    # Hide X and Y axes label marks
    ax.xaxis.set_tick_params(labelbottom=False)
    ax.yaxis.set_tick_params(labelleft=False)
    ax.zaxis.set_tick_params(labelright=False)

    # Hide X and Y axes tick marks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])


# colour maps
colours = ['cool', 'plasma']

# plot the time embedding 
cebra.plot_embedding(embedding=time_embedding[left,:], embedding_labels=trial_labels[left],ax=ax1, markersize=2, title='Time embedding', cmap=colours[0])
cebra.plot_embedding(embedding=time_embedding[right,:], embedding_labels=trial_labels[right],ax=ax1, markersize=2, title='Time embedding, AUC: 0.56', cmap=colours[1])

# plot the behaviour embedding 
cebra.plot_embedding(embedding=behaviour_embedding[left,:], embedding_labels=trial_labels[left],ax=ax2, markersize=2, title='Behaviour embedding', cmap=colours[0],)
cebra.plot_embedding(embedding=behaviour_embedding[right,:], embedding_labels=trial_labels[right],ax=ax2,markersize=2, title='Behaviour embedding, AUC: 0.87',  cmap=colours[1])


# Adjust the subplot layout manually
#plt.subplots_adjust(left=0.095, right=0.1, top=0.95, bottom=0.05, wspace=0.001)

# Adjust the subplot layout manually
plt.subplots_adjust(left=0.00001, right=0.55, top=0.95, bottom=0.05, wspace=0.0001)

# Use tight_layout with padding to ensure labels are not cut off


# Adjust label positions using bbox
ax2.set_zlabel("latent 3", labelpad=1, fontsize=10, bbox=dict(facecolor='white', edgecolor='none', pad=0.5))


Text(0.5, 0, 'latent 3')

From this pair of embeddings we can see that there is a clustering of the left (light blue) and right (purple) trials when we use CEBRA behaviour. 
What if we tried using both the (left/right) choice and the number of licks in either direction.

In [74]:
n_licks_ = n_licks.reshape(n_licks.shape[0],-1)
right_ = right.reshape(right.shape[0],-1).astype(np.float64)
left_ = left.reshape(left.shape[0], -1).astype(np.float64)

In [75]:
print(n_licks_.shape)
print(right_.shape)
print(left_.shape)

(1717, 1)
(1717, 1)
(1717, 1)


In [76]:
right

array([False, False, False, ..., False, False, False])

In [77]:
# arrange the labels like in the hippocampus recordings experiment demo
lr_licks = np.concatenate((n_licks_,left_,right_), axis=1)
lr_licks.shape

(1717, 3)

In [78]:
# build, train and compute with the time and behaviour models with this new labels
t_embed,b_embed =  cp.build_train_compute(formatted_nms,lr_licks)

pos:  0.0509 neg:  5.4406 total:  5.4915 temperature:  1.0000: 100%|██████████| 2000/2000 [00:13<00:00, 148.49it/s]
pos:  0.4388 neg:  5.5478 total:  5.9866 temperature:  1.0000: 100%|██████████| 2000/2000 [00:14<00:00, 139.02it/s]


In [79]:
np.unique(lr_licks[:,0])

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 21., 26.])

In [80]:
# define a function to view the embeddings
def view(time_embedding, behaviour_embedding, labels, label_classes, size=5):
 
    # create a figure and make the plots
    fig = plt.figure(figsize=(14,8))
    gs = gridspec.GridSpec(1, 2, figure=fig)


    ax81 = fig.add_subplot(gs[0,0], projection='3d')
    ax82 = fig.add_subplot(gs[0,1], projection='3d')
 


    # colour maps
    colours = ['cool', 'plasma', 'spring']

    # plot the time embedding 
    cebra.plot_embedding(embedding=time_embedding[label_classes[0],:], embedding_labels=labels[label_classes[0],0],ax=ax81, markersize=size, title='Time embedding', cmap=colours[0])
    cebra.plot_embedding(embedding=time_embedding[label_classes[1],:], embedding_labels=labels[label_classes[1],0],ax=ax81, markersize=size, title='Time embedding', cmap=colours[1])


    # plot the behaviour embedding 
    cebra.plot_embedding(embedding=behaviour_embedding[label_classes[0],:], embedding_labels=labels[label_classes[0],0],ax=ax82, markersize=size, title='Behaviour embedding', cmap=colours[0],)
    cebra.plot_embedding(embedding=behaviour_embedding[label_classes[1],:], embedding_labels=labels[label_classes[1],0],ax=ax82,markersize=size, title='Behaviour embedding',  cmap=colours[1])

    gs.tight_layout(figure=fig)

In [81]:
# define the classes
lr_classes = [left,right]
lr_classes[0].shape

(1717,)

In [82]:
view(t_embed, b_embed,lr_licks, lr_classes)

try this with the infoNCE setting -- maybe the behaviour embedding will make sense then

## Individual NMs

Now let's see how well the individual NMs capture the left/right choices.

In [83]:
individual_nms = cp.individual_datasets(traces_=traces)

shape of formatted array: (218572, 1)
shape of formatted array: (218572, 1)
shape of formatted array: (218572, 1)
shape of formatted array: (218572, 1)


In [84]:
b_embeds, t_embeds, t_labels, [positive, negative] = cp.nm_analysis_2(individual_nms, df_trials_ses, trace_times, choice_times, title=" ",label='choice')

neural tensor shape:  torch.Size([1717, 20])
reward labels shape:  (1717,)
choice labels shape:  (1717,)
rpe labels shape: (1717,)


pos:  0.1554 neg:  5.6496 total:  5.8050 temperature:  1.0000: 100%|██████████| 2000/2000 [00:12<00:00, 159.63it/s]
pos:  0.0003 neg:  6.2380 total:  6.2383 temperature:  1.0000: 100%|██████████| 2000/2000 [00:12<00:00, 154.06it/s]


COMPLETED ANALYSIS OF NM 0: 
neural tensor shape:  torch.Size([1717, 20])
reward labels shape:  (1717,)
choice labels shape:  (1717,)
rpe labels shape: (1717,)


pos:  0.0005 neg:  6.2362 total:  6.2367 temperature:  1.0000: 100%|██████████| 2000/2000 [00:13<00:00, 150.68it/s]
pos:  0.1276 neg:  6.0913 total:  6.2189 temperature:  1.0000: 100%|██████████| 2000/2000 [00:14<00:00, 134.10it/s]


COMPLETED ANALYSIS OF NM 1: 
neural tensor shape:  torch.Size([1717, 20])
reward labels shape:  (1717,)
choice labels shape:  (1717,)
rpe labels shape: (1717,)


pos:  0.3709 neg:  5.4750 total:  5.8459 temperature:  1.0000: 100%|██████████| 2000/2000 [00:14<00:00, 137.77it/s]
pos:  0.0002 neg:  6.2381 total:  6.2383 temperature:  1.0000: 100%|██████████| 2000/2000 [00:18<00:00, 106.06it/s]


COMPLETED ANALYSIS OF NM 2: 
neural tensor shape:  torch.Size([1717, 20])
reward labels shape:  (1717,)
choice labels shape:  (1717,)
rpe labels shape: (1717,)


pos:  0.1615 neg:  5.4783 total:  5.6397 temperature:  1.0000: 100%|██████████| 2000/2000 [00:13<00:00, 146.71it/s]
pos:  0.0001 neg:  6.2382 total:  6.2383 temperature:  1.0000: 100%|██████████| 2000/2000 [00:14<00:00, 136.18it/s]

COMPLETED ANALYSIS OF NM 3: 


In [85]:
len(b_embeds)

4

In [86]:
means, sds = cp.get_auc(b_embeds, t_labels)

In [87]:
means

[0.5016057788481418, 0.7046652101272329, 0.5, 0.5]

In [88]:
# first make function to make the plots given a list of embeddings
def plot4_embeddings(embeddings, labels , l_class, means=None, titles=['DA only', 'NE only', '5HT only', 'ACh only'], t=""):

    # number of plots
    n_plots = len(embeddings)

    n_columns = 2
    n_rows = n_plots//n_columns

    # create axis
    fig = plt.figure(figsize=(8,4*n_plots))
    gs = gridspec.GridSpec(n_rows, n_columns, figure=fig)

    # colour 
    c = ['cool','plasma','pink','winter']

    for i, embed in enumerate(embeddings):

        # create the axes
        ax = fig.add_subplot(gs[i // n_columns, i%n_columns], projection='3d')

        ax.set_xlabel("latent 1", labelpad=0.001, fontsize=13)
        ax.set_ylabel("latent 2", labelpad=0.001, fontsize=13)
        ax.set_zlabel("latent 3", labelpad=0.001, fontsize=13)

        # Hide X and Y axes label marks
        ax.xaxis.set_tick_params(labelbottom=False)
        ax.yaxis.set_tick_params(labelleft=False)
        ax.zaxis.set_tick_params(labelright=False)

        # Hide X and Y axes tick marks
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])


        if means.any():
            titles=['DA only, AUC:{}'.format(means[0]),'NE only, AUC:{}'.format(means[1]), '5HT only, AUC:{}'.format(means[2]), 'ACh only, AUC:{}'.format(means[3])]


        # plot the embedding
        cebra.plot_embedding(embedding=embed[l_class[0],:], embedding_labels=labels[l_class[0]], ax=ax, markersize=2,title=titles[i], cmap=c[0])
        cebra.plot_embedding(embedding=embed[l_class[1],:], embedding_labels=labels[l_class[1]], ax=ax, markersize=2,title=titles[i], cmap=c[1])

    plt.suptitle(t, fontsize=15)
    plt.tight_layout()

In [89]:
np.round(means,2)

array([0.5, 0.7, 0.5, 0.5])

In [90]:
plot4_embeddings(b_embeds, t_labels, [positive, negative], means=np.round(means,2))